In [15]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import os
import csv
from datetime import datetime
import pymysql

In [16]:
courses = {"19AD701" : ["BDA","Priyadharshini"],
           "19AD702" : ["IoT","Shenbagarajan"],
           "19AD751" : ["BDA Lab","Priyadharshini"],
           "19AD752" : ["IoT Lab","Shenbagarajan"],
           "19ADP15" : ["SNA", "Nirmalan"],
           "19ADC15" : ["CC", "Angela Jenefa Sujana"],
           "19ADC23" : ["BuDA","Nirmalan"]}

In [17]:
periods = {"Thursday" : ["19AD701","19AD702","19AD751","19AD752","19ADP15","19ADC15","19AD701","19ADC23"]}

In [18]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [19]:
dataset = datasets.ImageFolder('../Images/Class/')
idx_to_class = {i: c for c, i in dataset.class_to_idx.items()}

In [20]:
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = []
name_list = []
embedding_list = []

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True)
    if face is not None and prob > 0.90:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])


In [21]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt')

In [22]:
def get_current_datetime():
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
    day = now.strftime("%A")
    return date, time, day

In [23]:
def dbInsert(p,c,s,pn,d,t,da):
    connection = pymysql.connect(host="bxyjlxdzotlowe6fee2c-mysql.services.clever-cloud.com", port=3306, user="ugsqllp6usysqevp", passwd="E5t4uVTvTY65jaFIk5Lw", database="bxyjlxdzotlowe6fee2c")
    cursor = connection.cursor()
    insert1 = "INSERT INTO Attendance(PERIOD, COURSE, STAFF, STUDENT, DATE, TIME, DAY) VALUES(%s, %s, %s, %s, %s, %s ,%s);"
    data = (p,c,s,pn,d,t,da)

    cursor.execute(insert1,data)
    connection.commit()
    connection.close()

In [24]:
def predict_and_record(folder_path, data_path, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Image', 'Predicted Name', 'Distance', 'Date', 'Time', 'Day']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        saved_data = torch.load(data_path)
        embedding_list = saved_data[0]
        name_list = saved_data[1]

        for filename in os.listdir(folder_path):
            if filename.endswith(".png"):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path)
                face, prob = mtcnn(img, return_prob=True)
                emb = resnet(face.unsqueeze(0)).detach()

                dist_list = []
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                idx_min = dist_list.index(min(dist_list))
                predicted_name = name_list[idx_min]
                distance = min(dist_list)

                date, time, day = get_current_datetime()

                class_dir = os.path.join("../Images/Class/", predicted_name)

                img.save(os.path.join(class_dir, filename))

                writer.writerow({'Image': filename, 'Predicted Name': predicted_name, 'Distance': distance,
                                            'Date': date, 'Time': time, 'Day': day})
                
                if day=="Monday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Monday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Monday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Monday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Monday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Monday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Monday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Monday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Monday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                if day=="Tuesday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Tuesday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                if day=="Wednesday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Wednesday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                if day=="Thursday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Thursday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Thursday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Thursday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Thursday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Thursday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Thursday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Thursday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Thursday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                if day=="Friday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Friday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Saturday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Saturday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Saturday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Saturday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Saturday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Saturday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Saturday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                if day=="Saturday":
                    time_split = time.split(":")
                    if(int(time_split[0])==9):
                            period_list = periods.get("Friday")
                            p_id = period_list[0]
                            period = 1
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==10):
                            period_list = periods.get("Friday")
                            p_id = period_list[1]
                            period = 2
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==11):
                            period_list = periods.get("Friday")
                            p_id = period_list[2]
                            period = 3
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==12):
                            period_list = periods.get("Friday")
                            p_id = period_list[3]
                            period = 4
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==13):
                            period_list = periods.get("Friday")
                            p_id = period_list[4]
                            period = 5
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==14):
                            period_list = periods.get("Friday")
                            p_id = period_list[5]
                            period = 6
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==15):
                            period_list = periods.get("Friday")
                            p_id = period_list[6]
                            period = 7
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)

                    elif(int(time_split[0])==16):
                            period_list = periods.get("Friday")
                            p_id = period_list[7]
                            period = 8
                            sub,staff = courses.get(p_id)
                            dbInsert(period,p_id,staff,predicted_name,date,time,day)


In [25]:
folder_path = '../Images/output_images/testImages/'
data_path = 'data.pt'
output_csv = '../csvFolder/prediction.csv'

In [26]:
predict_and_record(folder_path, data_path, output_csv)